<a href="https://colab.research.google.com/github/peiva-git/deep_learning_project/blob/44-separare-i-notebook-in-file-separati/simple_ae_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports, definitions and setup

In [ ]:
!git clone https://github.com/peiva-git/deep_learning_project.git
%cd deep_learning_project

In [ ]:
import numpy as np
import dlproject as dlp
import matplotlib.pyplot as plt
import os.path

### Load the MNIST dataset

In [ ]:
# Uncomment one line to select the desired noise level

noise_level = 'high'
# noise_level = 'med'
# noise_level = 'low'

if noise_level == 'high':
  noise_value = 0.7
elif noise_level == 'med':
  noise_value = 0.4
elif noise_level == 'low':
  noise_value = 0.1

In [ ]:
dataset_builder = dlp.data.MNISTDatasetBuilder()
dataset_builder.preprocess_dataset_simple_ae(noise_value)
train_data, test_data = dataset_builder.train_x, dataset_builder.test_x
noisy_train_data, noisy_test_data = dataset_builder.noisy_train_data, dataset_builder.noisy_test_data

Display the first 10 noisy images.

In [ ]:
number_of_previews = 10
plt.figure(figsize=(20, 2))
for i in range(1, number_of_previews + 1):
    ax = plt.subplot(1, number_of_previews, i)
    plt.imshow(noisy_train_data[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Instantiate the model

In [ ]:
autoencoder_mnist = dlp.models.SimpleAutoencoder(input_shape=(28, 28, 1))
autoencoder_mnist.model.compile(optimizer='adam', loss='binary_crossentropy')
model = autoencoder_mnist.model

## Train the model

### Testing the model

First, we train the model to reconstruct the image that's given as an input. The reconstructed images should be similar, but not exactly the same.
We also save the model for later use.

In [ ]:
model.fit(
    x=train_data,
    y=train_data,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data)
)

Display the results.

In [ ]:
decoded_imgs = model.predict(test_data)

number_of_previews = 10
plt.figure(figsize=(20, 4))
for i in range(1, number_of_previews + 1):
    # Display original
    ax = plt.subplot(2, number_of_previews, i)
    plt.imshow(test_data[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, number_of_previews, i + number_of_previews)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Denoise images

Secondly, we retrain the model to reconstruct the image from a noisy input.

In [ ]:
from tensorflow.keras.saving import load_model

model_path = os.path.join(os.getcwd(), 'models',
                               f'{model.name}_mnist_{noise_level}_noise.keras')

if os.path.exists(model_path):
  model = load_model(model_path)
else:
  if not os.path.exists(os.path.join(os.getcwd(), 'models')):
    os.mkdir(os.path.join(os.getcwd(), 'models'))

  model.fit(
      x=noisy_train_data,
      y=train_data,
      epochs=100,
      batch_size=128,
      shuffle=True,
      validation_data=(noisy_test_data, test_data)
  )

model.save(os.path.join('models', f'{model.name}_mnist_{noise_level}_noise.keras'))

Let's take a look at the results. Top, the ground truth digits fed to the network, than the noisy version and finally the digits are reconstructed by the network. It seems to work pretty well.

In [ ]:
dlp.data.visualize.display_random_images(test_data, noisy_test_data, model.predict(noisy_test_data))

Compute predictions for the entire noisy MNIST dataset.

Compute PSNR and SSIM for model evaluation.

In [ ]:
reconstructed_images = model.predict(noisy_test_data)
print(dlp.evaluation.compute_mean_psnr(test_data, reconstructed_images))
print(dlp.evaluation.compute_mean_ssim(test_data, reconstructed_images))

313/313 [==============================] - 1s 2ms/step
65.75077366207717
0.9991426
